In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('train.csv')
df

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26565,26565,E,158.95,material_7,material_6,6,9,6,16,4,...,NaN,12.177,17.942,10.112,15.795,18.572,16.144,NaN,729.131,0
26566,26566,E,146.02,material_7,material_6,6,9,10,12,8,...,11.242,14.179,20.564,10.234,14.450,14.322,13.146,16.471,853.924,0
26567,26567,E,115.62,material_7,material_6,6,9,1,10,1,...,11.407,16.437,17.476,8.668,15.069,16.599,15.590,14.065,750.364,0
26568,26568,E,106.38,material_7,material_6,6,9,2,9,4,...,11.392,17.064,17.814,14.928,16.273,15.485,13.624,12.865,730.156,0


In [3]:
# fill all nan values with each columns mean
df.fillna(df.mean(), inplace=True)

C:\Users\hasan\AppData\Local\Temp/ipykernel_9776/1317108851.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.fillna(df.mean(), inplace=True)


In [6]:
df["product_code"] = df["product_code"].astype('category')
df["product_code"] = df["product_code"].cat.codes

df["attribute_0"] = df["attribute_0"].astype('category')
df["attribute_0"] = df["attribute_0"].cat.codes

df["attribute_1"] = df["attribute_1"].astype('category')
df["attribute_1"] = df["attribute_1"].cat.codes

df["attribute_2"] = df["attribute_2"].astype('category')
df["attribute_2"] = df["attribute_2"].cat.codes

df["attribute_3"] = df["attribute_3"].astype('category')
df["attribute_3"] = df["attribute_3"].cat.codes

df["failure"] = df["failure"].astype('category')

In [7]:
X = df.drop(["id", 'failure'], axis=1)
y = df['failure']

In [24]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

In [70]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, ...)

In [71]:
from sklearn.metrics import classification_report
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.76      0.86      0.81      4229
           1       0.83      0.72      0.77      4140

    accuracy                           0.79      8369
   macro avg       0.80      0.79      0.79      8369
weighted avg       0.80      0.79      0.79      8369



In [72]:
df_test = pd.read_csv('test.csv')
df

,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,0,119.57,0,1,0,0,6,9,6,19.305,...,18.654,10.802,15.909000,18.070000,13.772,13.659,16.825,13.742,17.710000,634.612
1,0,113.51,0,1,0,0,11,8,0,17.883,...,19.368,12.032,13.998000,18.846571,12.473,17.468,16.708,14.776,14.102000,537.037
2,0,112.16,0,1,0,0,8,12,4,18.475,...,17.774,11.743,17.046000,18.086000,10.907,13.363,15.737,17.065,16.021000,658.995
3,0,112.72,0,1,0,0,8,11,10,16.518,...,18.948,11.790,18.165000,16.163000,10.933,15.501,15.667,12.620,16.111000,594.301
4,0,208.00,0,1,0,0,14,16,8,17.808,...,19.141,12.370,14.578000,17.849000,11.941,16.070,16.183,13.324,17.150000,801.044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20770,3,144.74,1,0,2,1,0,4,9,18.465,...,18.573,11.691,16.123986,19.771000,11.562,17.246,15.131,15.209,16.636052,696.466
20771,3,74.53,1,0,2,1,4,8,7,18.900,...,17.917,10.980,16.027000,15.694000,13.564,15.494,15.296,13.812,16.501000,613.249
20772,3,67.73,1,0,2,1,10,11,2,18.656,...,19.630,10.436,16.137000,20.612000,11.134,16.519,15.525,14.175,17.728000,783.349
20773,3,126.15,1,0,2,1,8,16,11,16.536,...,19.575,12.672,15.422000,19.496000,9.319,15.817,17.403,16.437,15.179000,745.210


In [73]:
df = df_test.drop(["id"], axis=1)

df["product_code"] = df["product_code"].astype('category')
df["product_code"] = df["product_code"].cat.codes

df["attribute_0"] = df["attribute_0"].astype('category')
df["attribute_0"] = df["attribute_0"].cat.codes

df["attribute_1"] = df["attribute_1"].astype('category')
df["attribute_1"] = df["attribute_1"].cat.codes

df["attribute_2"] = df["attribute_2"].astype('category')
df["attribute_2"] = df["attribute_2"].cat.codes

df["attribute_3"] = df["attribute_3"].astype('category')
df["attribute_3"] = df["attribute_3"].cat.codes

df.fillna(df.mean(), inplace=True)

In [74]:
predicted = model.predict(df)
predicted

array([1, 1, 1, ..., 0, 0, 0])

In [75]:
results = pd.DataFrame({"id": df_test["id"], "failure": predicted})
results

,id,failure
0,26570,1
1,26571,1
2,26572,1
3,26573,1
4,26574,1
...,...,...
20770,47340,0
20771,47341,0
20772,47342,0
20773,47343,0


In [76]:
results.failure.value_counts()

1    12064
0     8711
Name: failure, dtype: int64

In [77]:
results.to_csv('results/result5.csv', index=False)